In [2]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
#import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
#from sklearn.metrics import classification_report, log_loss, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
import tensorflow as tf
import datetime
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
class_names = ['glioma','meningioma','notumor','pituitary']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (224, 224)

In [4]:
def load_data():
    """
        Load the data:
            - 14,034 images to train the network.
            - 3,000 images to evaluate how accurately the network learned to classify images.
    """
    TrainF=r"archive/Training"
    TestF=r"archive/Testing"
    datasets =  [TrainF,TestF]
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [5]:
(x_train, y_train), (x_test, y_test) = load_data()

Loading archive/Training


100%|██████████| 1339/1339 [00:01<00:00, 816.57it/s]


Loading archive/Testing


100%|██████████| 306/306 [00:00<00:00, 882.90it/s]


In [5]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [6]:
tumor_dir=r'archive/Training/glioma'
tumor_dir1=r'archive/Training/meningioma'
tumor_dir2=r'archive/Training/pituitary'
healthy_dir=r'archive/Training/notumor'
filepaths = []
labels= []
dict_list = [tumor_dir,tumor_dir1, tumor_dir2, healthy_dir]
for i, j in enumerate(dict_list):
    flist=os.listdir(j)
    for f in flist:
        fpath=os.path.join(j,f)
        filepaths.append(fpath)
        if i==0:
          labels.append('glioma_tumor')
        elif i==1:
          labels.append('meningioma_tumor')
        elif i==2:
          labels.append('pituitary_tumor')
        else:
          labels.append('no_tumor')
    
Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
tumor_data = pd.concat([Fseries,Lseries], axis=1)
tumor_df = pd.DataFrame(tumor_data)
print(tumor_df.head())
print(tumor_df["labels"].value_counts())

#shape of datatset
tumor_df.shape

                                filepaths        labels
0  archive/Training/glioma/Tr-gl_1135.jpg  glioma_tumor
1  archive/Training/glioma/Tr-gl_0559.jpg  glioma_tumor
2  archive/Training/glioma/Tr-gl_1121.jpg  glioma_tumor
3  archive/Training/glioma/Tr-gl_0217.jpg  glioma_tumor
4  archive/Training/glioma/Tr-gl_1109.jpg  glioma_tumor
labels
no_tumor            1595
pituitary_tumor     1457
meningioma_tumor    1339
glioma_tumor        1321
Name: count, dtype: int64


(5712, 2)

In [7]:
from sklearn.model_selection import train_test_split 


In [8]:
train_images, test_images = train_test_split(tumor_df, test_size=0.15, random_state=42)
train_set, val_set = train_test_split(tumor_df, test_size=0.2, random_state=42)

In [10]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(244,244),
                                      color_mode='rgb',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=32,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_images,x_col="filepaths", y_col="labels",
                                     target_size=(244,244),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=32,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=32,
                                    shuffle=False
                                   )

Found 4569 validated image filenames belonging to 4 classes.
Found 857 validated image filenames belonging to 4 classes.
Found 1143 validated image filenames belonging to 4 classes.


In [6]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch

/var/folders/wy/6rhcvjdx1d7fwb4sjvbnmlrh0000gn/T/ipykernel_2117/765264065.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [8]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
from tensorboard.plugins.hparams import api as hp

In [20]:
HP_NUM_FILTERS = hp.HParam('num_filters', hp.Discrete([16, 32, 64]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_FILTERS, HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [21]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(hparams[HP_NUM_FILTERS], 3, activation=tf.nn.relu,
    input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(hparams[HP_NUM_FILTERS], 3, activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=10) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [22]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [23]:
session_num = 0
for num_filters in HP_NUM_FILTERS.domain.values:
  for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
      for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_NUM_FILTERS: num_filters,
            HP_NUM_UNITS: num_units,
            HP_DROPOUT: dropout_rate,
            HP_OPTIMIZER: optimizer,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1


--- Starting trial: run-0
{'num_filters': 16, 'num_units': 64, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10


2023-06-07 12:19:11.409707: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


179/179 [==============================] - 22s 118ms/step - loss: 0.6465 - accuracy: 0.7463
Epoch 2/10
179/179 [==============================] - 21s 118ms/step - loss: 0.2640 - accuracy: 0.9023
Epoch 3/10
179/179 [==============================] - 21s 117ms/step - loss: 0.1515 - accuracy: 0.9473
Epoch 4/10
179/179 [==============================] - 21s 118ms/step - loss: 0.0748 - accuracy: 0.9762
Epoch 5/10
179/179 [==============================] - 21s 119ms/step - loss: 0.0468 - accuracy: 0.9858
Epoch 6/10
179/179 [==============================] - 22s 122ms/step - loss: 0.0255 - accuracy: 0.9932
Epoch 7/10
179/179 [==============================] - 21s 118ms/step - loss: 0.0224 - accuracy: 0.9935
Epoch 8/10
179/179 [==============================] - 21s 118ms/step - loss: 0.0103 - accuracy: 0.9972
Epoch 9/10
179/179 [==============================] - 21s 120ms/step - loss: 0.0153 - accuracy: 0.9963
Epoch 10/10
 1/41 [..............................] - ETA: 10s - loss: 9.3046e-04 - a

2023-06-07 12:22:44.849466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


41/41 [==============================] - 2s 51ms/step - loss: 0.2032 - accuracy: 0.9504
--- Starting trial: run-1
{'num_filters': 16, 'num_units': 64, 'dropout': 0.1, 'optimizer': 'sgd'}
Epoch 1/10


...

41/41 [==============================] - 9s 221ms/step - loss: 0.2330 - accuracy: 0.9497
--- Starting trial: run-35
{'num_filters': 64, 'num_units': 256, 'dropout': 0.2, 'optimizer': 'sgd'}
Epoch 1/10


2023-06-07 23:05:53.570092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


179/179 [==============================] - 78s 432ms/step - loss: 0.9395 - accuracy: 0.6021
Epoch 2/10
179/179 [==============================] - 77s 429ms/step - loss: 0.6000 - accuracy: 0.7724
Epoch 3/10
179/179 [==============================] - 82s 460ms/step - loss: 0.4693 - accuracy: 0.8239
Epoch 4/10
179/179 [==============================] - 79s 442ms/step - loss: 0.4033 - accuracy: 0.8480
Epoch 5/10
179/179 [==============================] - 88s 492ms/step - loss: 0.3315 - accuracy: 0.8775
Epoch 6/10
179/179 [==============================] - 81s 454ms/step - loss: 0.2746 - accuracy: 0.9002
Epoch 7/10
179/179 [==============================] - 88s 489ms/step - loss: 0.2264 - accuracy: 0.9196
Epoch 8/10
179/179 [==============================] - 90s 502ms/step - loss: 0.1909 - accuracy: 0.9343
Epoch 9/10
179/179 [==============================] - 97s 539ms/step - loss: 0.1663 - accuracy: 0.9424
Epoch 10/10
179/179 [==============================] - 87s 484ms/step - loss: 0.1360

2023-06-07 23:20:00.978931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


41/41 [==============================] - 8s 197ms/step - loss: 0.2226 - accuracy: 0.9077


In [24]:
%tensorboard --logdir logs/hparam_tuning